<a href="https://colab.research.google.com/github/madeleinegates/Transportation-Modeling-RNN/blob/main/Initialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.python.client import device_lib
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd 
train = pd.read_csv('/content/drive/My Drive/train.csv')

In [8]:
!ls

drive  sample_data


In [19]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import dask.dataframe as dd
import dask.array as da 

# Setting seed for reproducibility
np.random.seed(1234)  
PYTHONHASHSEED = 0

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM

# define path to save model
model_path = 'binary_model.h5'

ImportError: ignored

In [13]:
train.head()

,Unnamed: 0,trip_id,household_id,person_n,trip_n,travel_mode,bus_scale,survey_year,year,month,...,child_fare,free_fare,over_16_fare,no_car,under_one_car,at_least_one_car,snow,no_rain,light_rain,rain
0,1,0,0,0,0,drive,1.0,1,2012,4,...,0,0,0,0,0,1,0,1,0,0
1,2,1,0,0,1,drive,1.0,1,2012,4,...,0,0,0,0,0,1,0,1,0,0
2,3,2,0,0,2,drive,1.0,1,2012,4,...,0,0,0,0,0,1,0,1,0,0
3,4,3,0,0,3,drive,1.0,1,2012,4,...,0,0,0,0,0,1,0,1,0,0
4,5,4,0,1,2,drive,1.0,1,2012,4,...,0,0,0,0,0,1,0,1,0,0


In [18]:
train = train.categorize(columns = ["travel_mode"])

AttributeError: ignored

In [ ]:
# pick a window size
sequence_length = 5

# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # for each individual put rows in a single matrix
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    # Iterate over two lists in parallel.
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]
        
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

# function to generate labels
def gen_labels(id_df, seq_length, label):
    # labels in a single matrix for a given individual
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    # I have to remove the first seq_length labels
    # because for one id the first sequence of seq_length size have as target
    # the last label (the previus ones are discarded).
    # All the next id's sequences will have associated step by step one label as target. 
    return data_matrix[seq_length:num_elements, :]

# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

# The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. 
# Dropout is also applied after each LSTM layer to control overfitting. 
# Final layer uses softmax activation since this is a multi-classification problem.
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


# fit the network
history = model.fit(seq_array, label_array, epochs=100, batch_size=200, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 50, 100)           50400     
_________________________________________________________________
dropout_11 (Dropout)         (None, 50, 100)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dropout_12 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 80,651
Trainable params: 80,651
Non-trainable params: 0
_________________________________________________________________
None
Train on 14849 samples, validate on 782 samples
Epoch 1/100
 - 21s - loss: 0.2637 - acc: 0.8910 - val_loss: 0.1170 - val_acc: 0.9476


In [ ]:
# Definitely a work in progress...
class ChoiceRNN(torch.nn.Module):
    def __init__(self,n_x,n_i,n_y,k):
        super(ChoiceRNN,self).__init__()
        self.embed_user = torch.nn.Embedding(n_i+1,k,padding_idx=n_i)
        self.embed_item = torch.nn.Embedding(n_y+1,k,padding_idx=n_y)
        self.linear1 = torch.nn.Linear(n_x+k*2+1+1,10,bias=False)
        self.linear2 = torch.nn.Linear(10,8,bias=False)
        self.linear3 = torch.nn.Linear(8,1,bias=False)
        self.lsmax = torch.nn.LogSoftmax(0)

    def forward(self,x,idx):
        e_user = self.embed_user(idx[:,:,0].long())
        e_item = self.embed_item(idx[:,:,1].long())
        interaction = torch.einsum('bcij,bcj->bci',(e_user.view(e_user.size(0),e_user.size(1),1,e_user.size(2)),e_item))
        input_data = torch.cat((x,e_user,e_item,interaction),dim=2)
        input_data = input_data.view(input_data.size(1),input_data.size(2))
        a0  = torch.zeros(1)
        activations = torch.ones(input_data.size(0))
        for i in range(input_data.size(0)):
            x = torch.cat((input_data[i,:],a0))
            x = F.relu(self.linear1(x))
            x = F.relu(self.linear2(x))
            a0 = self.lsmax(self.linear3(x))
            activations[i] = a0
        result = self.lsmax(activations)
        return result.view(1,result.size(0))